# SCRAPING PLAYERS PERFORMANCE DATA FOR 2000-2023 SEASONS FROM https://fbref.com/en/ 

In [3]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import pickle
import os

In [4]:
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns=100
plt.style.use('ggplot')
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'}

In [ ]:
for i in range(2000,2023):
    season=f'{i}-{i+1}'
    epl_url=f'https://fbref.com/en/comps/9/{i}-{i+1}/{i}-{i+1}-Premier-League-Stats'
    bundesliga_url=f'https://fbref.com/en/comps/20/{i}-{i+1}/{i}-{i+1}-Bundesliga-Stats'
    laliga_url=f'https://fbref.com/en/comps/12/{i}-{i+1}/{i}-{i+1}-La-Liga-Stats'
    ligue1_url=f'https://fbref.com/en/comps/13/{i}-{i+1}/{i}-{i+1}-Ligue-1-Stats'
    seriaa_url=f'https://fbref.com/en/comps/11/{i}-{i+1}/{i}-{i+1}-Serie-A-Stats'
    ere_divisie=f'https://fbref.com/en/comps/23/{i}-{i+1}/{i}-{i+1}-Eredivisie-Stats'
    primeira_liga=f'https://fbref.com/en/comps/32/{i}-{i+1}/{i}-{i+1}-Primeira-Liga-Stats'

    league_urls=[epl_url,bundesliga_url,laliga_url,ligue1_url,seriaa_url,ere_divisie,primeira_liga]
    leagues=['EPL','Bundesliga','LaLiga','Ligue1','SeriaA','EreDivisie','PrimeiraLiga']
    ids=['91','201','121','131','111','231','321']
    
    leagues_dfs={}

    for league,league_url,id_num in zip(leagues,league_urls,ids):
        main_url=league_url
        html=requests.get(main_url).content
        soup=bs(html,'html.parser')

        table=soup.select_one(f'table[id="results{season}{id_num}_overall"] tbody')
        squad_links=[link['href'] for link in table.select('tr td a') if 'squad' in str(link)]
        squads=[link.text for link in table.select('tr td a') if 'squad' in str(link)]
        squad_dfs={}
        for squad, link in zip(squads,squad_links):
            url=f'https://fbref.com{link}'
            url=requests.get(url,headers=headers).content
            try:
                standard_stats=pd.read_html(url, match='Standard Stats')[0]
            except ValueError:
                standard_stats=None
            try:
                goalkeeping=pd.read_html(url, match='Goalkeeping')[0]
            except ValueError:
                goalkeeping=None
            try:
                advanced_goalkeeping=pd.read_html(url, match='Advanced Goalkeeping')[0]
            except ValueError:
                advanced_goalkeeping=None
            try:
                shooting=pd.read_html(url, match='Shooting')[0]
            except ValueError:
                shooting=None
            try:
                passing=pd.read_html(url, match='Passing')[0]
            except ValueError:
                passing=None
            try:
                goal_and_shot_creation=pd.read_html(url, match='Goal and Shot Creation')[0]
            except ValueError:
                goal_and_shot_creation=None
            try:
                defensive_actions=pd.read_html(url, match='Defensive Actions')[0]
            except ValueError:
                defensive_actions=None
            try:
                possetion=pd.read_html(url, match='Possession')[0]
            except ValueError:
                possetion=None
            try:
                playing_time=pd.read_html(url, match='Playing Time')[0]
            except ValueError:
                playing_time=None
            try:
                miscellaneous_stats=pd.read_html(url, match='Miscellaneous Stats')[0]
            except ValueError:
                miscellaneous_stats=None
            squad_dfs[squad]={
                'standard_stats':standard_stats,
                'goalkeeping':goalkeeping,
                'advanced_goalkeeping':advanced_goalkeeping,
                'shooting':shooting,
                'passing':passing,
                'goal_and_shot_creation':goal_and_shot_creation,
                'defensive_actions':defensive_actions,
                'possetion':possetion,
                'playing_time':playing_time,
                'miscellaneous_stats':miscellaneous_stats
            }
            leagues_dfs[league]=squad_dfs
            print(season,league,squad)
            
    #save as pickle file
    pickle.dump(leagues_dfs,open(f'leagues_dfs {season}.pickle', 'wb'))

# COMPILING ALL SEASONS DATA INTO A SINGLE DICTIONARY 

In [5]:
pickles=[]
seasons=[]

path=r'C:\Users\berid\python\players\\'

for file in os.listdir(path):
    if file.endswith('.pickle'):
        season=file.split('.pickle')[0].split(' ')[-1].strip()
        seasons.append(season)
        pickle_df=pickle.load(open(path+file,'rb'))
        pickles.append(pickle_df)

In [6]:
players_dict={season:pickle for season,pickle in zip(seasons,pickles)}

# CREATING DATAFRAMES

In [7]:
%%time

standard_stats=pd.DataFrame()
goalkeeping=pd.DataFrame()
advanced_goalkeeping=pd.DataFrame()
shooting=pd.DataFrame()
passing=pd.DataFrame()
goal_and_shot_creation=pd.DataFrame()
defensive_actions=pd.DataFrame()
possession=pd.DataFrame()
playing_time=pd.DataFrame()
miscellaneous_stats=pd.DataFrame()

for season in players_dict:
    league_dict=players_dict[season]
    for league in league_dict:
        team_dict=league_dict[league]
        for team in team_dict:
            values=team_dict[team]
            for value in values:
                if value =='standard_stats':
                    df=values[value]
                    try:
                        df.insert(0,'Season',season)
                        df.insert(1,'League',league)
                        df.insert(2,'Team',team)
                        standard_stats=pd.concat([standard_stats,df],ignore_index=True)
                    except AttributeError:
                        None
                    
                elif value =='goalkeeping':
                    df=values[value]
                    try:
                        df.insert(0,'Season',season)
                        df.insert(1,'League',league)
                        df.insert(2,'Team',team)
                        goalkeeping=pd.concat([goalkeeping,df],ignore_index=True)
                    except AttributeError:
                        None
                    
                elif value =='advanced_goalkeeping':
                    df=values[value]
                    try:
                        df.insert(0,'Season',season)
                        df.insert(1,'League',league)
                        df.insert(2,'Team',team)
                        advanced_goalkeeping=pd.concat([advanced_goalkeeping,df],ignore_index=True)
                    except AttributeError:
                        None
                    
                elif value =='shooting':
                    df=values[value]
                    try:
                        df.insert(0,'Season',season)
                        df.insert(1,'League',league)
                        df.insert(2,'Team',team)
                        shooting=pd.concat([shooting,df],ignore_index=True)
                    except AttributeError:
                        None
                    
                elif value =='passing':
                    df=values[value]
                    try:
                        df.insert(0,'Season',season)
                        df.insert(1,'League',league)
                        df.insert(2,'Team',team)
                        passing=pd.concat([passing,df],ignore_index=True)
                    except AttributeError:
                        None
                    
                elif value =='goal_and_shot_creation':
                    df=values[value]
                    try:
                        df.insert(0,'Season',season)
                        df.insert(1,'League',league)
                        df.insert(2,'Team',team)
                        goal_and_shot_creation=pd.concat([goal_and_shot_creation,df],ignore_index=True)
                    except AttributeError:
                        None
                    
                elif value =='defensive_actions':
                    df=values[value]
                    try:
                        df.insert(0,'Season',season)
                        df.insert(1,'League',league)
                        df.insert(2,'Team',team)
                        defensive_actions=pd.concat([defensive_actions,df],ignore_index=True)
                    except AttributeError:
                        None
                    
                elif value =='possetion':
                    df=values[value]
                    try:
                        df.insert(0,'Season',season)
                        df.insert(1,'League',league)
                        df.insert(2,'Team',team)
                        possession=pd.concat([possession,df],ignore_index=True)
                    except AttributeError:
                        None
                    
                elif value =='playing_time':
                    df=values[value]
                    try:
                        df.insert(0,'Season',season)
                        df.insert(1,'League',league)
                        df.insert(2,'Team',team)
                        playing_time=pd.concat([playing_time,df],ignore_index=True)
                    except AttributeError:
                        None
                    
                elif value =='miscellaneous_stats':
                    df=values[value]
                    try:
                        df.insert(0,'Season',season)
                        df.insert(1,'League',league)
                        df.insert(2,'Team',team)
                        miscellaneous_stats=pd.concat([miscellaneous_stats,df],ignore_index=True)
                    except AttributeError:
                        None
                    print(season,league,team)

2000-2001 EPL Manchester Utd
2000-2001 EPL Arsenal
2000-2001 EPL Liverpool
2000-2001 EPL Leeds United
2000-2001 EPL Ipswich Town
2000-2001 EPL Chelsea
2000-2001 EPL Sunderland
2000-2001 EPL Aston Villa
2000-2001 EPL Charlton Ath
2000-2001 EPL Southampton
2000-2001 EPL Newcastle Utd
2000-2001 EPL Tottenham
2000-2001 EPL Leicester City
2000-2001 EPL Middlesbrough
2000-2001 EPL West Ham
2000-2001 EPL Everton
2000-2001 EPL Derby County
2000-2001 EPL Manchester City
2000-2001 EPL Coventry City
2000-2001 EPL Bradford City
2000-2001 Bundesliga Bayern Munich
2000-2001 Bundesliga Schalke 04
2000-2001 Bundesliga Dortmund
2000-2001 Bundesliga Leverkusen
2000-2001 Bundesliga Hertha BSC
2000-2001 Bundesliga Freiburg
2000-2001 Bundesliga Werder Bremen
2000-2001 Bundesliga Kaiserslautern
2000-2001 Bundesliga Wolfsburg
2000-2001 Bundesliga Köln
2000-2001 Bundesliga 1860 Munich
2000-2001 Bundesliga Hansa Rostock
2000-2001 Bundesliga Hamburger SV
2000-2001 Bundesliga Energie Cottbus
2000-2001 Bundesliga

2002-2003 Bundesliga Kaiserslautern
2002-2003 Bundesliga Leverkusen
2002-2003 Bundesliga Arminia
2002-2003 Bundesliga Nürnberg
2002-2003 Bundesliga Energie Cottbus
2002-2003 LaLiga Real Madrid
2002-2003 LaLiga Real Sociedad
2002-2003 LaLiga La Coruña
2002-2003 LaLiga Celta Vigo
2002-2003 LaLiga Valencia
2002-2003 LaLiga Barcelona
2002-2003 LaLiga Athletic Club
2002-2003 LaLiga Betis
2002-2003 LaLiga Mallorca
2002-2003 LaLiga Sevilla
2002-2003 LaLiga Osasuna
2002-2003 LaLiga Atlético Madrid
2002-2003 LaLiga Málaga
2002-2003 LaLiga Valladolid
2002-2003 LaLiga Villarreal
2002-2003 LaLiga Racing Sant
2002-2003 LaLiga Espanyol
2002-2003 LaLiga Recreativo
2002-2003 LaLiga Alavés
2002-2003 LaLiga Rayo Vallecano
2002-2003 Ligue1 Lyon
2002-2003 Ligue1 Monaco
2002-2003 Ligue1 Marseille
2002-2003 Ligue1 Bordeaux
2002-2003 Ligue1 Sochaux
2002-2003 Ligue1 Auxerre
2002-2003 Ligue1 Guingamp
2002-2003 Ligue1 Lens
2002-2003 Ligue1 Nantes
2002-2003 Ligue1 Nice
2002-2003 Ligue1 Paris S-G
2002-2003 Ligue1

2004-2005 Ligue1 Paris S-G
2004-2005 Ligue1 Sochaux
2004-2005 Ligue1 Strasbourg
2004-2005 Ligue1 Nice
2004-2005 Ligue1 Toulouse
2004-2005 Ligue1 Ajaccio
2004-2005 Ligue1 Bordeaux
2004-2005 Ligue1 Metz
2004-2005 Ligue1 Nantes
2004-2005 Ligue1 Caen
2004-2005 Ligue1 Bastia
2004-2005 Ligue1 Istres
2004-2005 SeriaA Juventus
2004-2005 SeriaA Milan
2004-2005 SeriaA Inter
2004-2005 SeriaA Udinese
2004-2005 SeriaA Sampdoria
2004-2005 SeriaA Palermo
2004-2005 SeriaA Messina
2004-2005 SeriaA Roma
2004-2005 SeriaA Livorno
2004-2005 SeriaA Reggina
2004-2005 SeriaA Lecce
2004-2005 SeriaA Cagliari
2004-2005 SeriaA Lazio
2004-2005 SeriaA Siena
2004-2005 SeriaA Chievo
2004-2005 SeriaA Fiorentina
2004-2005 SeriaA Parma
2004-2005 SeriaA Bologna
2004-2005 SeriaA Brescia
2004-2005 SeriaA Atalanta
2004-2005 EreDivisie PSV Eindhoven
2004-2005 EreDivisie Ajax
2004-2005 EreDivisie AZ Alkmaar
2004-2005 EreDivisie Feyenoord
2004-2005 EreDivisie Heerenveen
2004-2005 EreDivisie Twente
2004-2005 EreDivisie Vitesse


2006-2007 EreDivisie Ajax
2006-2007 EreDivisie AZ Alkmaar
2006-2007 EreDivisie Twente
2006-2007 EreDivisie Heerenveen
2006-2007 EreDivisie Roda JC
2006-2007 EreDivisie Feyenoord
2006-2007 EreDivisie Groningen
2006-2007 EreDivisie Utrecht
2006-2007 EreDivisie NEC Nijmegen
2006-2007 EreDivisie NAC Breda
2006-2007 EreDivisie Vitesse
2006-2007 EreDivisie Sparta R'dam
2006-2007 EreDivisie Heracles Almelo
2006-2007 EreDivisie Willem II
2006-2007 EreDivisie Excelsior
2006-2007 EreDivisie RKC Waalwijk
2006-2007 EreDivisie ADO Den Haag
2006-2007 PrimeiraLiga Porto
2006-2007 PrimeiraLiga Sporting CP
2006-2007 PrimeiraLiga Benfica
2006-2007 PrimeiraLiga Braga
2006-2007 PrimeiraLiga Belenenses
2006-2007 PrimeiraLiga Paços
2006-2007 PrimeiraLiga União de Leiria
2006-2007 PrimeiraLiga Nacional
2006-2007 PrimeiraLiga Estrela
2006-2007 PrimeiraLiga Boavista
2006-2007 PrimeiraLiga Marítimo
2006-2007 PrimeiraLiga Naval
2006-2007 PrimeiraLiga Académica
2006-2007 PrimeiraLiga Vitória Setúbal
2006-2007 Pri

2008-2009 PrimeiraLiga Naval
2008-2009 PrimeiraLiga Vitória Setúbal
2008-2009 PrimeiraLiga Belenenses
2008-2009 PrimeiraLiga Trofense
2009-2010 EPL Chelsea
2009-2010 EPL Manchester Utd
2009-2010 EPL Arsenal
2009-2010 EPL Tottenham
2009-2010 EPL Manchester City
2009-2010 EPL Aston Villa
2009-2010 EPL Liverpool
2009-2010 EPL Everton
2009-2010 EPL Birmingham City
2009-2010 EPL Blackburn
2009-2010 EPL Stoke City
2009-2010 EPL Fulham
2009-2010 EPL Sunderland
2009-2010 EPL Bolton
2009-2010 EPL Wolves
2009-2010 EPL Wigan Athletic
2009-2010 EPL West Ham
2009-2010 EPL Burnley
2009-2010 EPL Hull City
2009-2010 EPL Portsmouth
2009-2010 Bundesliga Bayern Munich
2009-2010 Bundesliga Schalke 04
2009-2010 Bundesliga Werder Bremen
2009-2010 Bundesliga Leverkusen
2009-2010 Bundesliga Dortmund
2009-2010 Bundesliga Stuttgart
2009-2010 Bundesliga Hamburger SV
2009-2010 Bundesliga Wolfsburg
2009-2010 Bundesliga Mainz 05
2009-2010 Bundesliga Eint Frankfurt
2009-2010 Bundesliga Hoffenheim
2009-2010 Bundeslig

...

2013-2014 Ligue1 Paris S-G
2013-2014 Ligue1 Monaco
2013-2014 Ligue1 Lille
2013-2014 Ligue1 Saint-Étienne
2013-2014 Ligue1 Lyon
2013-2014 Ligue1 Marseille
2013-2014 Ligue1 Bordeaux
2013-2014 Ligue1 Lorient
2013-2014 Ligue1 Toulouse
2013-2014 Ligue1 Bastia
2013-2014 Ligue1 Reims
2013-2014 Ligue1 Rennes
2013-2014 Ligue1 Nantes
2013-2014 Ligue1 Evian
2013-2014 Ligue1 Montpellier
2013-2014 Ligue1 Guingamp
2013-2014 Ligue1 Nice
2013-2014 Ligue1 Sochaux
2013-2014 Ligue1 Valenciennes
2013-2014 Ligue1 Ajaccio
2013-2014 SeriaA Juventus
2013-2014 SeriaA Roma
2013-2014 SeriaA Napoli
2013-2014 SeriaA Fiorentina
2013-2014 SeriaA Inter
2013-2014 SeriaA Parma
2013-2014 SeriaA Torino
2013-2014 SeriaA Milan
2013-2014 SeriaA Lazio
2013-2014 SeriaA Hellas Verona
2013-2014 SeriaA Atalanta
2013-2014 SeriaA Sampdoria
2013-2014 SeriaA Udinese
2013-2014 SeriaA Genoa
2013-2014 SeriaA Cagliari
2013-2014 SeriaA Chievo
2013-2014 SeriaA Sassuolo
2013-2014 SeriaA Catania
2013-2014 SeriaA Bologna
2013-2014 SeriaA Liv

2015-2016 SeriaA Torino
2015-2016 SeriaA Atalanta
2015-2016 SeriaA Bologna
2015-2016 SeriaA Sampdoria
2015-2016 SeriaA Palermo
2015-2016 SeriaA Udinese
2015-2016 SeriaA Carpi
2015-2016 SeriaA Frosinone
2015-2016 SeriaA Hellas Verona
2015-2016 EreDivisie PSV Eindhoven
2015-2016 EreDivisie Ajax
2015-2016 EreDivisie Feyenoord
2015-2016 EreDivisie AZ Alkmaar
2015-2016 EreDivisie Utrecht
2015-2016 EreDivisie Heracles Almelo
2015-2016 EreDivisie Groningen
2015-2016 EreDivisie Zwolle
2015-2016 EreDivisie Vitesse
2015-2016 EreDivisie NEC Nijmegen
2015-2016 EreDivisie ADO Den Haag
2015-2016 EreDivisie Twente
2015-2016 EreDivisie Heerenveen
2015-2016 EreDivisie Roda JC
2015-2016 EreDivisie Excelsior
2015-2016 EreDivisie Willem II
2015-2016 EreDivisie De Graafschap
2015-2016 EreDivisie Cambuur
2015-2016 PrimeiraLiga Benfica
2015-2016 PrimeiraLiga Sporting CP
2015-2016 PrimeiraLiga Porto
2015-2016 PrimeiraLiga Braga
2015-2016 PrimeiraLiga Arouca
2015-2016 PrimeiraLiga Rio Ave
2015-2016 PrimeiraLig

2017-2018 EreDivisie Twente
2017-2018 PrimeiraLiga Porto
2017-2018 PrimeiraLiga Benfica
2017-2018 PrimeiraLiga Sporting CP
2017-2018 PrimeiraLiga Braga
2017-2018 PrimeiraLiga Rio Ave
2017-2018 PrimeiraLiga Chaves
2017-2018 PrimeiraLiga Marítimo
2017-2018 PrimeiraLiga Boavista
2017-2018 PrimeiraLiga Vitória
2017-2018 PrimeiraLiga Portimonense
2017-2018 PrimeiraLiga Tondela
2017-2018 PrimeiraLiga Belenenses
2017-2018 PrimeiraLiga Aves
2017-2018 PrimeiraLiga Vitória Setúbal
2017-2018 PrimeiraLiga Moreirense
2017-2018 PrimeiraLiga Feirense
2017-2018 PrimeiraLiga Paços
2017-2018 PrimeiraLiga Estoril
2018-2019 EPL Manchester City
2018-2019 EPL Liverpool
2018-2019 EPL Chelsea
2018-2019 EPL Tottenham
2018-2019 EPL Arsenal
2018-2019 EPL Manchester Utd
2018-2019 EPL Wolves
2018-2019 EPL Everton
2018-2019 EPL Leicester City
2018-2019 EPL West Ham
2018-2019 EPL Watford
2018-2019 EPL Crystal Palace
2018-2019 EPL Newcastle Utd
2018-2019 EPL Bournemouth
2018-2019 EPL Burnley
2018-2019 EPL Southampton

2020-2021 EPL West Ham
2020-2021 EPL Tottenham
2020-2021 EPL Arsenal
2020-2021 EPL Leeds United
2020-2021 EPL Everton
2020-2021 EPL Aston Villa
2020-2021 EPL Newcastle Utd
2020-2021 EPL Wolves
2020-2021 EPL Crystal Palace
2020-2021 EPL Southampton
2020-2021 EPL Brighton
2020-2021 EPL Burnley
2020-2021 EPL Fulham
2020-2021 EPL West Brom
2020-2021 EPL Sheffield Utd
2020-2021 Bundesliga Bayern Munich
2020-2021 Bundesliga RB Leipzig
2020-2021 Bundesliga Dortmund
2020-2021 Bundesliga Wolfsburg
2020-2021 Bundesliga Eint Frankfurt
2020-2021 Bundesliga Leverkusen
2020-2021 Bundesliga Union Berlin
2020-2021 Bundesliga M'Gladbach
2020-2021 Bundesliga Stuttgart
2020-2021 Bundesliga Freiburg
2020-2021 Bundesliga Hoffenheim
2020-2021 Bundesliga Mainz 05
2020-2021 Bundesliga Augsburg
2020-2021 Bundesliga Hertha BSC
2020-2021 Bundesliga Arminia
2020-2021 Bundesliga Köln
2020-2021 Bundesliga Werder Bremen
2020-2021 Bundesliga Schalke 04
2020-2021 LaLiga Atlético Madrid
2020-2021 LaLiga Real Madrid
202

2022-2023 Bundesliga Köln
2022-2023 Bundesliga Augsburg
2022-2023 Bundesliga Bochum
2022-2023 Bundesliga Hertha BSC
2022-2023 Bundesliga Stuttgart
2022-2023 Bundesliga Schalke 04
2022-2023 Bundesliga Hoffenheim
2022-2023 LaLiga Barcelona
2022-2023 LaLiga Real Madrid
2022-2023 LaLiga Atlético Madrid
2022-2023 LaLiga Real Sociedad
2022-2023 LaLiga Betis
2022-2023 LaLiga Villarreal
2022-2023 LaLiga Rayo Vallecano
2022-2023 LaLiga Osasuna
2022-2023 LaLiga Athletic Club
2022-2023 LaLiga Mallorca
2022-2023 LaLiga Celta Vigo
2022-2023 LaLiga Girona
2022-2023 LaLiga Sevilla
2022-2023 LaLiga Valladolid
2022-2023 LaLiga Espanyol
2022-2023 LaLiga Cádiz
2022-2023 LaLiga Valencia
2022-2023 LaLiga Getafe
2022-2023 LaLiga Almería
2022-2023 LaLiga Elche
2022-2023 Ligue1 Paris S-G
2022-2023 Ligue1 Marseille
2022-2023 Ligue1 Lens
2022-2023 Ligue1 Monaco
2022-2023 Ligue1 Rennes
2022-2023 Ligue1 Lille
2022-2023 Ligue1 Nice
2022-2023 Ligue1 Reims
2022-2023 Ligue1 Lorient
2022-2023 Ligue1 Lyon
2022-2023 Lig

# CHANGING COLUMN NAMES 

In [9]:
tables=[standard_stats,
goalkeeping,
advanced_goalkeeping,
shooting,
passing,
goal_and_shot_creation,
defensive_actions,
possession,
playing_time,
miscellaneous_stats]


table_names=['standard_stats',
'goalkeeping',
'advanced_goalkeeping',
'shooting',
'passing',
'goal_and_shot_creation',
'defensive_actions',
'possession',
'playing_time',
'miscellaneous_stats']

for table in tables:
    table.columns=[col[0]+'_'+col[1] if col[1]!='' else col[0] for col in table.columns]

# CREATING CSV FILES FOR EACH DATAFRAME 

In [11]:
path=r'C:\Users\berid\python\players\\'


for table,table_name in zip(tables,table_names):
    table.to_csv(path+f'{table_name}.csv',index=False)